In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24769


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24764,2021-06-21,47,沖縄県,33,20134,0,167
24765,2021-06-22,47,沖縄県,98,20232,4,171
24766,2021-06-23,47,沖縄県,96,20328,0,171
24767,2021-06-24,47,沖縄県,62,20390,4,175
24768,2021-06-25,47,沖縄県,82,20472,4,179


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11594,9181,150
43,香川県,19499,2090,30
44,高知県,20553,1777,24
45,鳥取県,16337,466,2
46,鹿児島県,24242,3647,38


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24762,2021-06-19,47,沖縄県,97,20040,0,167,660.0
24763,2021-06-20,47,沖縄県,61,20101,0,167,617.0
24764,2021-06-21,47,沖縄県,33,20134,0,167,596.0
24765,2021-06-22,47,沖縄県,98,20232,4,171,587.0
24766,2021-06-23,47,沖縄県,96,20328,0,171,568.0
24767,2021-06-24,47,沖縄県,62,20390,4,175,533.0
24768,2021-06-25,47,沖縄県,82,20472,4,179,529.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24759,2021-06-16,47,沖縄県,115,19760,0,163,848.0,0.074766,0.021168,-0.339080
24760,2021-06-17,47,沖縄県,97,19857,4,167,779.0,-0.156522,0.005376,-0.415663
24761,2021-06-18,47,沖縄県,86,19943,0,167,720.0,-0.113402,0.007248,-0.406897
24762,2021-06-19,47,沖縄県,97,20040,0,167,660.0,0.127907,0.013697,-0.382166
24763,2021-06-20,47,沖縄県,61,20101,0,167,617.0,-0.371134,0.008904,-0.413462
24764,2021-06-21,47,沖縄県,33,20134,0,167,596.0,-0.459016,0.012012,-0.388889
24765,2021-06-22,47,沖縄県,98,20232,4,171,587.0,1.969697,0.153185,-0.084112
24766,2021-06-23,47,沖縄県,96,20328,0,171,568.0,-0.020408,0.139589,-0.165217
24767,2021-06-24,47,沖縄県,62,20390,4,175,533.0,-0.354167,0.111354,-0.360825
24768,2021-06-25,47,沖縄県,82,20472,4,179,529.0,0.322581,0.173637,-0.046512


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
526,2021-06-25,北海道,33,4,299.0,0.269231,-0.048903,-0.565789
1053,2021-06-25,青森県,2,0,13.0,-0.750000,NaN,inf
1580,2021-06-25,岩手県,4,0,32.0,-0.600000,NaN,-0.200000
2107,2021-06-25,宮城県,8,0,38.0,0.333333,0.325510,1.000000
2634,2021-06-25,秋田県,12,0,40.0,1.400000,NaN,inf


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12648,5238,110,2021-06-25,22,0,61.0,1.000000,0.754553,0.571429
1,京都府,13702,16519,241,2021-06-25,10,1,86.0,0.000000,1.453940,-0.444444
2,佐賀県,21607,2553,24,2021-06-25,0,0,11.0,-1.000000,NaN,NaN
3,兵庫県,14756,40816,1279,2021-06-25,20,1,136.0,0.250000,0.001865,-0.444444
4,北海道,527,41169,1377,2021-06-25,33,4,299.0,0.269231,-0.048903,-0.565789


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12648,5238,110,2021-06-25,22,0,61.0,1.000000,0.754553,0.571429,1813859
1,京都府,13702,16519,241,2021-06-25,10,1,86.0,0.000000,1.453940,-0.444444,2545899
2,佐賀県,21607,2553,24,2021-06-25,0,0,11.0,-1.000000,NaN,NaN,823810
3,兵庫県,14756,40816,1279,2021-06-25,20,1,136.0,0.250000,0.001865,-0.444444,5549568
4,北海道,527,41169,1377,2021-06-25,33,4,299.0,0.269231,-0.048903,-0.565789,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,527,41169,1377,2021-06-25,33,4,299.0,0.269231,-0.048903,-0.565789,5267762,78.152734,781.527336,5.676035
41,青森県,1054,2465,31,2021-06-25,2,0,13.0,-0.750000,NaN,inf,1275783,19.321468,193.214677,1.018982
19,岩手県,1581,1657,46,2021-06-25,4,0,32.0,-0.600000,NaN,-0.200000,1235517,13.411390,134.113897,2.590009
11,宮城県,2108,9099,88,2021-06-25,8,0,38.0,0.333333,0.325510,1.000000,2292385,39.692286,396.922855,1.657662
36,秋田県,2635,804,20,2021-06-25,12,0,40.0,1.400000,NaN,inf,985416,8.158991,81.589907,4.059199
15,山形県,3162,2020,47,2021-06-25,0,0,3.0,NaN,NaN,NaN,1082296,18.664025,186.640254,0.277188
35,福島県,3689,4823,161,2021-06-25,9,0,69.0,-0.181818,0.016167,-0.307692,1881981,25.627251,256.272513,3.666349
38,茨城県,4216,10409,162,2021-06-25,56,0,188.0,0.473684,0.515288,1.666667,2921436,35.629738,356.297383,6.435191
27,栃木県,4743,6902,81,2021-06-25,27,0,140.0,0.421053,0.088171,0.173913,1965516,35.115461,351.154608,7.122812
37,群馬県,5270,8017,152,2021-06-25,4,0,21.0,inf,NaN,1.000000,1969439,40.707024,407.070237,1.066293


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16796.489362
std       30842.399553
min         466.000000
25%        2509.000000
50%        5012.000000
75%       10936.000000
max      171507.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6851,171507,2218,2021-06-25,562,2,3186.0,-0.014035,0.089465,0.240618,13834925,123.966700,1239.667002,23.028676
1,大阪府,14229,102950,2640,2021-06-25,120,5,727.0,0.034483,0.204939,0.518987,8849635,116.332482,1163.324815,8.215028
2,神奈川県,7378,66374,944,2021-06-25,221,2,1255.0,0.151042,0.008388,-0.043290,9209442,72.071685,720.716847,13.627319
3,愛知県,12121,50884,950,2021-06-25,66,2,544.0,-0.340000,0.101619,-0.377358,7575530,67.168898,671.688978,7.181016
4,埼玉県,5797,46001,830,2021-06-25,100,1,624.0,-0.159664,0.105725,0.562500,7390054,62.247177,622.471771,8.443781
5,北海道,527,41169,1377,2021-06-25,33,4,299.0,0.269231,-0.048903,-0.565789,5267762,78.152734,781.527336,5.676035
6,兵庫県,14756,40816,1279,2021-06-25,20,1,136.0,0.250000,0.001865,-0.444444,5549568,73.548067,735.480672,2.450641
7,千葉県,6324,39703,703,2021-06-25,126,1,767.0,-0.023256,0.024944,-0.066667,6319772,62.823469,628.234689,12.136514
8,福岡県,21080,35376,518,2021-06-25,37,2,241.0,0.027778,0.172169,0.423077,5129841,68.961202,689.612017,4.698001
9,沖縄県,24769,20472,179,2021-06-25,82,4,529.0,0.322581,0.173637,-0.046512,1481547,138.179889,1381.798890,35.705921


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      40.512839
std       29.406304
min        8.111004
25%       22.324087
50%       30.324114
75%       52.686346
max      138.179889
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24769,20472,179,2021-06-25,82,4,529.0,0.322581,0.173637,-0.046512,1481547,138.179889,1381.798890,35.705921
26,東京都,6851,171507,2218,2021-06-25,562,2,3186.0,-0.014035,0.089465,0.240618,13834925,123.966700,1239.667002,23.028676
16,山梨県,10013,2073,21,2021-06-25,14,0,157.0,-0.391304,-0.035751,-0.600000,826579,25.079273,250.792725,18.993950
33,福井県,9486,1222,34,2021-06-25,17,0,134.0,-0.320000,0.198136,0.545455,780053,15.665602,156.656022,17.178320
32,神奈川県,7378,66374,944,2021-06-25,221,2,1255.0,0.151042,0.008388,-0.043290,9209442,72.071685,720.716847,13.627319
5,千葉県,6324,39703,703,2021-06-25,126,1,767.0,-0.023256,0.024944,-0.066667,6319772,62.823469,628.234689,12.136514
44,高知県,20553,1777,24,2021-06-25,9,0,72.0,0.800000,0.214116,0.500000,709230,25.055342,250.553417,10.151855
7,埼玉県,5797,46001,830,2021-06-25,100,1,624.0,-0.159664,0.105725,0.562500,7390054,62.247177,622.471771,8.443781
9,大阪府,14229,102950,2640,2021-06-25,120,5,727.0,0.034483,0.204939,0.518987,8849635,116.332482,1163.324815,8.215028
10,奈良県,15283,8113,129,2021-06-25,17,1,105.0,-0.055556,0.377852,0.214286,1353837,59.925973,599.259734,7.755734


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    32.000000
mean      0.291002
std       0.431569
min      -0.096773
25%       0.022750
50%       0.170732
75%       0.338596
max       1.850200
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
29,滋賀県,13175,5535,90,2021-06-25,11,0,61.0,0.833333,1.850200,1.750000,1420948,38.952868,389.528681,4.292909
1,京都府,13702,16519,241,2021-06-25,10,1,86.0,0.000000,1.453940,-0.444444,2545899,64.884742,648.847421,3.377982
13,富山県,8432,2034,37,2021-06-25,5,0,34.0,0.666667,0.961905,-0.666667,1055999,19.261382,192.613819,3.219700
0,三重県,12648,5238,110,2021-06-25,22,0,61.0,1.000000,0.754553,0.571429,1813859,28.877658,288.776581,3.362996
30,熊本県,22661,6470,114,2021-06-25,7,1,39.0,0.166667,0.606215,0.166667,1769880,36.556151,365.561507,2.203539
38,茨城県,4216,10409,162,2021-06-25,56,0,188.0,0.473684,0.515288,1.666667,2921436,35.629738,356.297383,6.435191
14,山口県,18445,3125,77,2021-06-25,6,1,30.0,2.000000,0.380272,0.200000,1369882,22.812184,228.121838,2.189970
10,奈良県,15283,8113,129,2021-06-25,17,1,105.0,-0.055556,0.377852,0.214286,1353837,59.925973,599.259734,7.755734
11,宮城県,2108,9099,88,2021-06-25,8,0,38.0,0.333333,0.325510,1.000000,2292385,39.692286,396.922855,1.657662
31,石川県,8959,3927,115,2021-06-25,1,0,26.0,-0.750000,0.301701,-0.857143,1139612,34.459097,344.590966,2.281478


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
21,広島県,17918,11463,174,2021-06-25,12,3,116.0,-0.333333,-0.096773,-0.600000,2826858,40.550321,405.503212,4.103496
4,北海道,527,41169,1377,2021-06-25,33,4,299.0,0.269231,-0.048903,-0.565789,5267762,78.152734,781.527336,5.676035
16,山梨県,10013,2073,21,2021-06-25,14,0,157.0,-0.391304,-0.035751,-0.600000,826579,25.079273,250.792725,18.993950
18,岡山県,17391,7584,126,2021-06-25,0,0,13.0,-1.000000,-0.023810,-1.000000,1903627,39.839738,398.397375,0.682907
46,鹿児島県,24242,3647,38,2021-06-25,2,0,17.0,0.000000,-0.015873,-0.777778,1630146,22.372229,223.722292,1.042851
3,兵庫県,14756,40816,1279,2021-06-25,20,1,136.0,0.250000,0.001865,-0.444444,5549568,73.548067,735.480672,2.450641
32,神奈川県,7378,66374,944,2021-06-25,221,2,1255.0,0.151042,0.008388,-0.043290,9209442,72.071685,720.716847,13.627319
35,福島県,3689,4823,161,2021-06-25,9,0,69.0,-0.181818,0.016167,-0.307692,1881981,25.627251,256.272513,3.666349
5,千葉県,6324,39703,703,2021-06-25,126,1,767.0,-0.023256,0.024944,-0.066667,6319772,62.823469,628.234689,12.136514
27,栃木県,4743,6902,81,2021-06-25,27,0,140.0,0.421053,0.088171,0.173913,1965516,35.115461,351.154608,7.122812


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.     21846.125  43226.25   64606.375  85986.5   107366.625
 128746.75  150126.875 171507.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  3.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 23.,  4.,  8.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="富山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
8422,2021-06-16,16,富山県,5,1974,0,37,31.0,0.000000,0.416327,-0.285714
8423,2021-06-17,16,富山県,11,1985,0,37,37.0,1.200000,0.628571,1.200000
8424,2021-06-18,16,富山県,15,2000,0,37,51.0,0.363636,0.794805,14.000000
8425,2021-06-19,16,富山県,6,2006,0,37,52.0,-0.600000,0.137662,0.200000
8426,2021-06-20,16,富山県,3,2009,0,37,50.0,-0.500000,0.066234,-0.400000
8427,2021-06-21,16,富山県,8,2017,0,37,53.0,1.666667,0.304329,0.600000
8428,2021-06-22,16,富山県,1,2018,0,37,49.0,-0.875000,0.179329,-0.800000
8429,2021-06-23,16,富山県,8,2026,0,37,52.0,7.000000,1.179329,0.600000
8430,2021-06-24,16,富山県,3,2029,0,37,44.0,-0.625000,0.918615,-0.727273
8431,2021-06-25,16,富山県,5,2034,0,37,34.0,0.666667,0.961905,-0.666667
